LSTM AutoEncoder Model to generate sequential data


In [ ]:
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed

In [10]:
#define input timeseries
timeseries = np.array([[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
                       [0.1**3, 0.2**3, 0.3**3, 0.4**3, 0.5**3, 0.6**3, 0.7**3, 0.8**3, 0.9**3]]).transpose()

timeseries.shape

(9, 2)

In [ ]:
timeseries = np.array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]).transpose()

In [11]:
timesteps = 3
stride=1
n_features = timeseries.shape[1]
timeseries

array([[0.1  , 0.001],
       [0.2  , 0.008],
       [0.3  , 0.027],
       [0.4  , 0.064],
       [0.5  , 0.125],
       [0.6  , 0.216],
       [0.7  , 0.343],
       [0.8  , 0.512],
       [0.9  , 0.729]])

In [14]:
def temporalize(ts, lookback):
    output_ts = []
    
    for i in range(len(ts)-lookback+1):
        t = []
        for j in range(lookback):
            # Gather past records upto the lookback period
            t.append(ts[(i+j)])
        output_ts.append(t)
        #output_y.append(y[i+lookback+1])
    return output_ts #output_y

In [15]:
X= temporalize(timeseries, timesteps)
X=np.array(X)
X.shape

(7, 3, 2)

In [16]:
X

array([[[0.1  , 0.001],
        [0.2  , 0.008],
        [0.3  , 0.027]],

       [[0.2  , 0.008],
        [0.3  , 0.027],
        [0.4  , 0.064]],

       [[0.3  , 0.027],
        [0.4  , 0.064],
        [0.5  , 0.125]],

       [[0.4  , 0.064],
        [0.5  , 0.125],
        [0.6  , 0.216]],

       [[0.5  , 0.125],
        [0.6  , 0.216],
        [0.7  , 0.343]],

       [[0.6  , 0.216],
        [0.7  , 0.343],
        [0.8  , 0.512]],

       [[0.7  , 0.343],
        [0.8  , 0.512],
        [0.9  , 0.729]]])

In [17]:
model = Sequential()
model.add(LSTM(128, activation='relu', input_shape=(timesteps,n_features), return_sequences=True))
model.add(LSTM(64, activation='relu', return_sequences=False))
model.add(RepeatVector(timesteps))
model.add(LSTM(64, activation='relu', return_sequences=True))
model.add(LSTM(128, activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(n_features)))
model.compile(optimizer='adam', loss='mse')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 3, 128)            67072     
                                                                 
 lstm_1 (LSTM)               (None, 64)                49408     
                                                                 
 repeat_vector (RepeatVector  (None, 3, 64)            0         
 )                                                               
                                                                 
 lstm_2 (LSTM)               (None, 3, 64)             33024     
                                                                 
 lstm_3 (LSTM)               (None, 3, 128)            98816     
                                                                 
 time_distributed (TimeDistr  (None, 3, 2)             258       
 ibuted)                                                

In [18]:
# fit model
model.fit(X, X, epochs=300, batch_size=5, verbose=0)
# demonstrate reconstruction
yhat = model.predict(X, verbose=0)
print('---Predicted---')
print(np.round(yhat,3))
print('---Actual---')
print(np.round(X, 3))

---Predicted---
[[[ 0.139 -0.001]
  [ 0.213  0.007]
  [ 0.291  0.027]]

 [[ 0.198  0.01 ]
  [ 0.298  0.028]
  [ 0.391  0.063]]

 [[ 0.285  0.029]
  [ 0.405  0.069]
  [ 0.502  0.127]]

 [[ 0.388  0.063]
  [ 0.507  0.132]
  [ 0.6    0.219]]

 [[ 0.499  0.12 ]
  [ 0.606  0.224]
  [ 0.697  0.345]]

 [[ 0.609  0.202]
  [ 0.7    0.344]
  [ 0.795  0.507]]

 [[ 0.697  0.347]
  [ 0.797  0.511]
  [ 0.897  0.732]]]
---Actual---
[[[0.1   0.001]
  [0.2   0.008]
  [0.3   0.027]]

 [[0.2   0.008]
  [0.3   0.027]
  [0.4   0.064]]

 [[0.3   0.027]
  [0.4   0.064]
  [0.5   0.125]]

 [[0.4   0.064]
  [0.5   0.125]
  [0.6   0.216]]

 [[0.5   0.125]
  [0.6   0.216]
  [0.7   0.343]]

 [[0.6   0.216]
  [0.7   0.343]
  [0.8   0.512]]

 [[0.7   0.343]
  [0.8   0.512]
  [0.9   0.729]]]
